In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import plot_model

class LeNet:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize the model
        model = Sequential()
        inputShape = (height, width, depth)
        
        # first set of CONV => RELU => POOL layers
        model.add(Conv2D(20, (5, 5), padding="same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        # second set of CONV => RELU => POOL layers
        model.add(Conv2D(50, (5, 5), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(500))
        model.add(Activation("relu"))
        
        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        
        return model

# Example usage:
model = LeNet.build(width=32, height=32, depth=3, classes=10)

# Generate a plot of the model
plot_model(model, to_file='lenet_architecture.png', show_shapes=True, show_layer_names=True)


In [2]:
#
# This network is based on the Line Follower Robot using CNN by Nawaz Ahmad
# towardsdatascience.com
#
from packaging import version
import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Flatten, Dense
from keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
if version.parse(tf.__version__) < version.parse("2.9.0"):
    from keras.preprocessing.image import img_to_array
else:
    from tensorflow.keras.utils import img_to_array
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.callbacks import TensorBoard
from imutils import paths
import numpy as np
import argparse
import random
import cv2
import os
from datetime import datetime

In [3]:
class LeNet:
  @staticmethod
  def build(width, height, depth, classes):
    # initialize the model
    model = Sequential()
    inputShape = (height, width, depth)
# first set of CONV => RELU => POOL layers
    model.add(Conv2D(20, (5, 5), padding="same",
      input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# second set of CONV => RELU => POOL layers
    model.add(Conv2D(50, (5, 5), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
# first (and only) set of FC => RELU layers
    model.add(Flatten())
    model.add(Dense(500))
    model.add(Activation("relu"))
# softmax classifier
    model.add(Dense(classes))
    model.add(Activation("softmax"))
# return the constructed network architecture
    return model

In [4]:
dataset = './trainImages/'

# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []

# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images(dataset)))
random.seed(42)
random.shuffle(imagePaths)
# loop over the input images
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (64, 64))
    image = img_to_array(image)
    data.append(image)
# extract the class label from the image path and update the
    # labels list
    label = imagePath.split(os.path.sep)[-2]
    # print(label)
    if label == 'left':
        label = 0
    elif label == 'forward':
        label = 1
    else:
        label =2
    labels.append(label)
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float32") / 255.0
labels = np.array(labels)

# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
    labels, test_size=0.25, random_state=42)
# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=3)
testY = to_categorical(testY, num_classes=3)

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logdir)

[INFO] loading images...


In [5]:
# initialize the number of epochs to train for, initial learning rate,
# and batch size
EPOCHS = 100
INIT_LR = 1e-3
BS = 32
# initialize the model
print("[INFO] compiling model...")
model = LeNet.build(width=64, height=64, depth=3, classes=3)

lr_schedule = ExponentialDecay(
    initial_learning_rate=INIT_LR,
    decay_steps=EPOCHS,
    decay_rate=0.96
)
opt = Adam(learning_rate=lr_schedule)

# model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] compiling model...


/Users/mafu/miniforge3/envs/ros_env_2/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-12-15 17:43:31.348811: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-12-15 17:43:31.348869: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-12-15 17:43:31.348874: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-12-15 17:43:31.348908: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-15 17:43:31.348930: I tensorflow/core/common_runtime/pluggable_device/pluggable_device

In [5]:
from tensorflow.keras.callbacks import EarlyStopping

# Early stopping callback
early_stopping = EarlyStopping(
    monitor='val_loss',       # Metric to monitor (e.g., validation loss)
    patience=10,              # Number of epochs with no improvement after which training will stop
    restore_best_weights=True # Restore model weights from the epoch with the best value of the monitored metric
)

# Training the model with early stopping
H = model.fit(
    trainX, trainY,
    batch_size=BS,
    validation_data=(testX, testY),
    epochs=EPOCHS,
    verbose=1,
    callbacks=[tensorboard_callback, early_stopping]
)

Epoch 1/100


2024-12-15 16:07:08.866854: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


810/810 ━━━━━━━━━━━━━━━━━━━━ 39s 46ms/step - accuracy: 0.8358 - loss: 0.4331 - val_accuracy: 0.8862 - val_loss: 0.2688
Epoch 2/100
810/810 ━━━━━━━━━━━━━━━━━━━━ 37s 45ms/step - accuracy: 0.8842 - loss: 0.2736 - val_accuracy: 0.8964 - val_loss: 0.2321
Epoch 3/100
810/810 ━━━━━━━━━━━━━━━━━━━━ 37s 45ms/step - accuracy: 0.9033 - loss: 0.2357 - val_accuracy: 0.8981 - val_loss: 0.2357
Epoch 4/100
810/810 ━━━━━━━━━━━━━━━━━━━━ 37s 46ms/step - accuracy: 0.9127 - loss: 0.2108 - val_accuracy: 0.9194 - val_loss: 0.1931
Epoch 5/100
810/810 ━━━━━━━━━━━━━━━━━━━━ 37s 46ms/step - accuracy: 0.9232 - loss: 0.1919 - val_accuracy: 0.9331 - val_loss: 0.1722
Epoch 6/100
810/810 ━━━━━━━━━━━━━━━━━━━━ 37s 46ms/step - accuracy: 0.9319 - loss: 0.1727 - val_accuracy: 0.9376 - val_loss: 0.1673
Epoch 7/100
810/810 ━━━━━━━━━━━━━━━━━━━━ 38s 46ms/step - accuracy: 0.9416 - loss: 0.1497 - val_accuracy: 0.9411 - val_loss: 0.1513
Epoch 8/100
810/810 ━━━━━━━━━━━━━━━━━━━━ 38s 47ms/step - accuracy: 0.9457 - loss: 0.1412 - val_

In [6]:
# save the model to disk
print("[INFO] serializing network...")
model.save('models_h5/self_driving_model.h5')

[INFO] serializing network...
